In [1]:
import pandas as pd

# Relative path to the CSV file
file_path = "../data/Zagreb/restaurant_list.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())


      Restaurant Title
0     Plješivička Klet
1              Vinodol
2           Pri Zvoncu
3  Batak Grill - Rudeš
4      Restoran Taurus


In [1]:
from src.scraping.browser_managment import BrowserManager, ParallelBrowserManager
from src.scraping.google_scraping.google_init import GoogleInit


In [2]:
def search_wrapper(query, city):
    # Assume the query is the restaurant or place name
    try:
        # Simulate a search operation by just combining the place name with the city
        result = f"Searching for {query} in {city}... Success"
        return result
    except Exception as exc:
        print(f"Error processing query '{query}': {exc}")
        return None  # Return None as a specific error indicator

city = 'Zagreb'
queries = ['Restoran Casablanca', 'Coffee Shop', 'Library']

In [3]:
parallel_manager = ParallelBrowserManager(workers=3)
results = parallel_manager.perform_parallel_action(search_wrapper, [(query, city) for query in queries])

for result in results:
    if result is not None:
        print(result)
    else:
        print("Search failed or returned no results.")

Task completed with result: Searching for Coffee Shop in Zagreb... Success
Task completed with result: Searching for Library in Zagreb... Success
Task completed with result: Searching for Restoran Casablanca in Zagreb... Success
Searching for Coffee Shop in Zagreb... Success
Searching for Library in Zagreb... Success
Searching for Restoran Casablanca in Zagreb... Success


In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time

def search_wrapper(query, city):
    try:
        # Setup Chrome options for Selenium
        chrome_options = Options()
        # Uncomment the next line if you want the browser to run headless
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Initialize the WebDriver
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

        # Construct the search query
        search_query = f"{query} {city}"

        # Navigate to Google and perform the search
        driver.get("http://www.google.com")
        search_box = driver.find_element("name", "q")
        search_box.send_keys(search_query + Keys.RETURN)  # Perform the search

        # Optionally, add logic here to parse the results

        # Let's just print the current URL after the search
        print(f"Performed search for '{search_query}', current URL: {driver.current_url}")

        # Close the browser
        return True
    except Exception as exc:
        print(f"Error during search for '{query} {city}': {exc}")
        return False

# Example usage
city = 'Zagreb'
queries = ['Restoran Casablanca', 'Coffee Shop', 'Library']
for query in queries:
    result = search_wrapper(query, city)
    print(f"Search for '{query}' in '{city}' completed with result: {result}")


Error during search for 'Restoran Casablanca Zagreb': Message: element not interactable
  (Session info: chrome=122.0.6261.70)
Stacktrace:
	GetHandleVerifier [0x00558CD3+51395]
	(No symbol) [0x004C5F31]
	(No symbol) [0x0037E004]
	(No symbol) [0x003B2548]
	(No symbol) [0x003B16A4]
	(No symbol) [0x003D3E8C]
	(No symbol) [0x003ADF7E]
	(No symbol) [0x003D4124]
	(No symbol) [0x003EA570]
	(No symbol) [0x003D3C26]
	(No symbol) [0x003AC629]
	(No symbol) [0x003AD40D]
	GetHandleVerifier [0x008D6493+3711107]
	GetHandleVerifier [0x0091587A+3970154]
	GetHandleVerifier [0x00910B68+3950424]
	GetHandleVerifier [0x00609CD9+776393]
	(No symbol) [0x004D1704]
	(No symbol) [0x004CC5E8]
	(No symbol) [0x004CC799]
	(No symbol) [0x004BDDC0]
	BaseThreadInitThunk [0x767F7BA9+25]
	RtlInitializeExceptionChain [0x7713BD2B+107]
	RtlClearBits [0x7713BCAF+191]
Search for 'Restoran Casablanca' in 'Zagreb' completed with result: False
Error during search for 'Coffee Shop Zagreb': Message: element not interactable
  (Ses